In [3]:
import os
import requests
import json

print (refresh_rwgps_routes())


Found 446 existing routes in directory 'tracks'...
Checking 25 routes from RWGPS for missing routes
['11764387', '11775438', '11776097', '12179465', '12671150', '12688340', '12688604', '12688676', '12688698', '12689060', '12689154', '13008233', '13014266', '13016694', '13456204', '13513246', '13513281', '13930011', '14001770', '14085886', '14490288', '14556011', '14698114', '14698353', '14698504', '14777365', '15991343', '15992257', '16486265', '16855624', '16944678', '17286115', '17419435', '17419790', '17540056', '17636490', '17637149', '17637327', '17858346', '17958460', '17999406', '17999469', '18384108', '18384169', '18726535', '18832720', '18863046', '18874850', '18875008', '18875133', '18875405', '18875715', '18876426', '18876655', '18888567', '18895184', '18895945', '18926319', '18967242', '19038800', '19204584', '19252546', '19778991', '19786977', '19787269', '19945279', '20208363', '20208477', '20209022', '20211004', '20286398', '20348462', '20714803', '20748723', '20836201',

In [2]:
# Load list of routes for a given user from RWGPS
# Downloads routes into directory ".\tracks" relative to CWD, only if the GPX file isn't already there
# If you have a developer token, this can check many routes, otherwise the public interface seems to load the most recent 25
def refresh_rwgps_routes ( directory = 'tracks', user:int = 657096, api_key = '', auth_token = '' ) -> list:

    if not(os.path.exists(directory)):
        os.mkdir(directory)

    # get list of existing routes
    files = []
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)) and file.endswith('.gpx') :
            files.append(file[:file.rfind(".")])
    print (f"Found {len(files)} existing routes in directory '{directory}'...")
    
    # Now, load the latest routes from RWGPS - public method seems to get latest 25 routes... User 657096 is me!
    if api_key == "":
        r = requests.get (f"https://ridewithgps.com/users/{user}/routes.json" )
    else:
        #...or call with credentials can take parameters, but need auth token...
        r = requests.get (f"https://ridewithgps.com/users/{user}/routes.json", params={"offset" : "0", "limit" : "500", "version": "2", "apikey": api_key, "auth_token": auth_token } )

    if (r.status_code == 200) :
        if (api_key == ""):
            routes = json.loads(r.content)
        else:
            # return structure is different in authenticated call!
            routes = json.loads(r.content)['results']
    else:
        print(f"Error: {r.status_code} - {r.content}")
        
    # Loop through the routes, and download if we don't have it
    print(f"Checking {len(routes)} routes from RWGPS for missing routes")
    for route in routes:
        id = route['id']
        if (f'{id}' not in files):
            print(f'Route: {id} does not exist - downloading GPX...')
            r = requests.get (f"https://ridewithgps.com/routes/{id}.gpx")
            if (r.status_code == 200):
                with open(f"tracks\\{id}.gpx", "wb") as file:
                    file.write(r.content)
                files.append( str(id) )
            else:
                print(f'Failed to get route {id}: {r.status_code}')
    
    return files



In [18]:
# Use Haversine formula to caucluate distance between 2 points in meters
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of Earth in meters
    R = 6371000

    # convert decimal degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance


In [26]:
import os
import requests
import json
import gpxpy
import folium
from math import sin, cos, sqrt, atan2, radians

# hard wire search criteria (Woodbridge) distance is in metres
lat, lon, dist = 52.16344964521758, 0.5069332657261196, 500

# refresh our list of tracks from RWGPS
files = 

# OK - now we loop through our tracks and look for one that's near our destination...
matched_routes = set() 	# a set to avoid duplicates when adding route matches...
print(f"Now checking for track that's within {dist}m of lat:{lat}, lon:{lon}")
num_tracks = len(os.listdir(directory))
count = 0
for file in os.listdir(directory):
	count += 1
	if os.path.isfile(os.path.join(directory, file)) and file.endswith('.gpx') :
		print(f"Checking files: {file} - {count / num_tracks:.1%}", end="\r")
		with open(os.path.join(directory, file), 'r') as gpx_file:
			gpx = gpxpy.parse(gpx_file)
			for track in gpx.tracks:
				for segment in track.segments:
					for point in segment.points:
						distance = calculate_distance(lat, lon, point.latitude, point.longitude)
						#print (f"lat:{point.latitude}, lon:{point.longitude}, distance:{distance}")
						if distance < dist:
							#print (f"Route {file} matched! - lat:{point.latitude}, lon:{point.longitude}, distance:{distance}")
							matched_routes.add(file[:file.rfind(".")])
			
print (f"Matched tracks: {matched_routes}")

# Now make a map with the selected routes on it...
map = folium.map()








SyntaxError: invalid syntax (3146297424.py, line 12)